In [5]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import joblib
import numpy as np

## Exploratory Data Analysis

In [8]:
df = pd.read_csv("../data/processed/transformed_data.csv")

## Pipeline / Basic Modeling

In [ ]:
# Load DataFrame
df = transformed_df.copy()

# Define input features and target features
input_features = [
    "championId", "matchupChampion", "individualPosition", 
    "kills", "deaths", "assists", 
    "goldEarned", "totalDamageDealt", 
    "totalDamageTaken", "totalHeal", "win"
]
target_features = [
    "Boots_id", "Legendary_1_id", "Legendary_2_id",
    "Keystone", "PrimarySlot1", "PrimarySlot2",
    "PrimarySlot3", "SecondarySlot1", "SecondarySlot2"
]

# One-Hot Encode individualPosition
one_hot_encoder = OneHotEncoder(drop='first', handle_unknown='ignore')

# Scale numerical features
scaler = StandardScaler()

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("position", one_hot_encoder, ["individualPosition"]),
        ("scaling", scaler, ["kills", "deaths", "assists", "goldEarned", "totalDamageDealt", "totalDamageTaken", "totalHeal"])
    ],
    remainder='passthrough'  # Keep championId, matchupChampion, win as-is
)

# Create pipeline for the entire data processing
pipeline = Pipeline([
    ("preprocessor", preprocessor)
])

In [ ]:
# Fit and transform the input features
X = df[input_features]
X_processed = pipeline.fit_transform(X)

# Target features (outputs)
y = df[target_features]

In [ ]:
# Initialize the base classifier
base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_classifier = MultiOutputClassifier(base_classifier)

# Fit the classifier
multi_target_classifier.fit(X_processed, y)

In [ ]:
def predict_optimal_build(champion_id, matchup_champion_id, df, pipeline, model):
    """
    Predict the optimal item build and runes for the given champion and matchup champion.

    Parameters:
    - champion_id: int, champion ID of the player.
    - matchup_champion_id: int, champion ID of the opponent.
    - df: DataFrame, original DataFrame with historical data.
    - pipeline: preprocessing pipeline used for transforming the features.
    - model: trained MultiOutputClassifier model.

    Returns:
    - DataFrame, containing the predicted items and runes.
    """
    # Create a new input DataFrame with average values for other features
    input_data = df[(df['championId'] == champion_id) & (df['matchupChampion'] == matchup_champion_id)].mean().to_dict()

    # Override champion-specific fields
    input_data['championId'] = champion_id
    input_data['matchupChampion'] = matchup_champion_id

    # Create a DataFrame for input
    input_features = df.columns.difference(target_features)
    input_df = pd.DataFrame([input_data], columns=input_features)

    # Preprocess the input features using the pipeline
    input_processed = pipeline.transform(input_df)

    # Predict the output
    predicted_output = model.predict(input_processed)

    # Convert the prediction to a DataFrame for better readability
    predicted_df = pd.DataFrame(predicted_output, columns=target_features)

    return predicted_df

# Define input champion IDs
champion_id = 24  # e.g., Jax
matchup_champion_id = 777  # e.g., Yone

# Call the prediction function
predicted_build = predict_optimal_build(champion_id, matchup_champion_id, df, pipeline, multi_target_classifier)

# Display the predicted items and runes
print("Predicted Items and Runes for the given matchup:")
print(predicted_build)